1. Install nugets

In [1]:
#r "nuget:Microsoft.ML,*-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22313.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22504.6 Microsoft.ML, 2.0.0-preview.22313.1

2. Add global usings

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.ML.Trainers;
using System.IO;
using System.Text;

3. Add input data model

In [1]:
public class SkabInput
{
	[LoadColumn(0)]
	public string TimeStamp {get;set;}

	[LoadColumn(5)]
	public float Tempurature {get;set;}

	[LoadColumn(9)]
	public float Anomaly {get;set;}
}

4. Add output data model

In [1]:
public class SkabOutput
{
	[ColumnName("PredictedLabel")]
	public bool IsAnomaly {get;set;}
}

5. Load the dataset and split for test and training 

In [1]:
var mlContext = new MLContext();
var trainingData = mlContext.Data.LoadFromTextFile<SkabInput>("../../../ML.NET.Demo/Assets/alldata_skab.csv", hasHeader: true, separatorChar: ',');
var dataSplit = mlContext.Data.TrainTestSplit(trainingData, testFraction: 0.2);

6. *Optional* Display dataset

In [1]:
dataSplit.TestSet.ToTabularDataResource().Display();

7. Train the model (Feature Engineering)

In [1]:
var options = new RandomizedPcaTrainer.Options()
            {
                FeatureColumnName = "Features",
                Rank = 1,
                Seed = 10,
            };

var model = mlContext.AnomalyDetection.Trainers.RandomizedPca(options);

var pipeline = mlContext.Transforms.Concatenate("Features", nameof(SkabInput.Tempurature))
					.Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"))
					.Append(mlContext.Transforms.CopyColumns("Label", nameof(SkabInput.Anomaly)))
					.Append(model);

var trainedModel = pipeline.Fit(dataSplit.TrainSet);

8. Evaluate

In [1]:
var testSet = trainedModel.Transform(dataSplit.TestSet);
var metrics = mlContext.AnomalyDetection.Evaluate(testSet);

metrics

AreaUnderRocCurve,DetectionRateAtFalsePositiveCount
0.49089878394621367,0


9. Save model

In [1]:
mlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, "./Anomaly.mdl");

10. Load the model and test

In [1]:
var newSample = new SkabInput
{
	Tempurature = 26.776f
};

using (var stream = new FileStream("./Anomaly.mdl", FileMode.Open, FileAccess.Read))
{
    var model = mlContext.Model.Load(stream, out var _);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<SkabInput, SkabOutput>(model);
    var results = predictionEngine.Predict(newSample);

    results.Display();
}

IsAnomaly
False
